# IMPORT

In [24]:
# Web Scraping
import requests
from bs4 import BeautifulSoup
# Data Analysis
import pandas as pd
import numpy as np
# Data Visualization

# Read Table into DataFrame

In [36]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0]
# remove Not assigned Borough
df = df[df['Borough'] != "Not assigned"]
# assign Borough tco NaN Neighbourhood
for i in range(len(df)):
    if df.iloc[i,2] == 'Not assigned':
        df.iloc[i,2] = df.iloc[i,1]
# combine areas with the same postal code
functions = {'Borough': 'sum', 'Neighbourhood': lambda column: ",".join(column)}
df_clean = df.groupby('Postal Code', as_index=False).aggregate(functions).reindex(columns=df.columns)
df_clean.head(10)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [39]:
df_clean.shape

(103, 3)